In [1]:
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook

## CN

In [6]:
folder_path = "C:\\Users\\yingyy\\Desktop\\Discoverability\\CN"

In [7]:
result_df = []

In [8]:
for f in os.listdir(folder_path):
    f_path = os.path.join(folder_path,f)
    df = pd.read_excel(f_path,sheet_name="Sheet1")
    result_df.append(df)

In [10]:
result= pd.concat(result_df)

In [14]:
result.head()

,gs_arc,ASIN,gl_desc,target_gv_band,target_glance_view_count,target_impression_count,gv/impression,specific,specific (Node ID),brand,imp_rank,gv_imp_rank,rank
0,DE-CN,B086R5Z41S,Wireless,E,11,162,0.067901,Cross/Self Service/Special Features Stores/par...,2129028071;1904552071;139706071;1809018071;187...,小米,112,136,248
1,DE-CN,B087MRJRL7,Wireless,E,7,282,0.024823,Cross/Self Service/Special Features Stores/par...,2129028071;1904552071;139706071;1809018071;187...,小米,139,59,198
2,DE-CN,B0885Z1PKP,Wireless,E,3,198,0.015152,Cross/Self Service/Special Features Stores/par...,2129028071;153985071;1904552071;139706071;1809...,小米,118,39,157
3,DE-CN,B08B99D7FJ,Wireless,D,160,1063,0.150517,Refinements/cn-home/颜色/蓝色 (2051210051);Cross/S...,2051210051;1904552071;1758245071;139706071;180...,小米,216,216,432
4,DE-CN,B08B9BD523,Wireless,D,127,2308,0.055026,Refinements/cn-home/颜色/蓝色 (2051210051);Cross/S...,2051210051;1904552071;1758245071;139706071;180...,小米,310,112,422


In [18]:
#create np percentile function for agg
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n,interpolation = 'midpoint')
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [19]:
result_1 = result.reset_index().groupby(['gl_desc','brand'])['target_glance_view_count'].agg({np.mean,np.size,np.min,np.max,np.median,np.var,percentile(25),percentile(75)}).rename(columns={np.mean:'gv_average',np.size:'size',np.min:'gv_min',np.max:'gv_max',np.median:'gv_median',np.var:'variance',percentile(25):'25%',percentile(75):'75%'})

In [21]:
result_1.reset_index().to_excel('result_1.xlsx', encoding = 'utf-8')

## Other MPs

In [2]:
MPs = ['CN','AU','DE','SG','UK','AE','SA']

In [8]:
MPs = ['AE','SA']

In [3]:
bins = [0,50,100,200,500,1000,2000,5000,10000,20000,50000,100000,200000,500000]

In [7]:
ExcelWorkbook = load_workbook(r"C:\Users\yingyy\Desktop\Discoverability\Brand_Analysis.xlsx")
writer = pd.ExcelWriter(r"C:\Users\yingyy\Desktop\Discoverability\Brand_Analysis.xlsx", engine = 'openpyxl')
writer.book = ExcelWorkbook

In [9]:
for mp in MPs:
    file_path = "C:\\Users\\yingyy\\Desktop\\Discoverability\\Raw_Data\\US-"+mp+"_sort_asin.xlsx"
    df = pd.read_excel(file_path,sheet_name="Sheet1")
    df_brand = pd.read_excel(r"C:\Users\yingyy\Desktop\Discoverability\Top Brands by 2020 GV.xlsx",sheet_name= mp)
    result_1 = df.reset_index().groupby(['gl_desc','brand'])['target_glance_view_count'].agg({np.mean,np.size,np.min,np.max,np.median,np.var}).rename(columns={np.mean:'gv_average',np.size:'size',np.min:'gv_min',np.max:'gv_max',np.median:'gv_median',np.var:'variance'})
    result_1 = result_1.reset_index()
    result_1['brand'] = result_1['brand'].astype(str)
    df_brand['Brand'] = df_brand['Brand'].astype(str)
    result = pd.merge(result_1,df_brand,left_on = 'brand', right_on = 'Brand',how='left')
    result.to_excel(writer, sheet_name = mp)
    result_2 = result[result['size']>=10]
    with open (r"C:\Users\yingyy\Desktop\Discoverability\New Text Document.txt",'a') as file:
        file.write(mp+":"+str(len(pd.unique(result_2['Brand']))))
    result_2 = result_2[['Brand','Rank']].drop_duplicates()
    result_2['Rank_binned'] = pd.cut(result_2['Rank'],bins)
    result_2['Rank_binned'].value_counts(normalize= False,sort = False).to_excel(writer, sheet_name = mp+'_BrandRank')

In [10]:
writer.save()
writer.close()

In [23]:
result.to_excel(r"C:\Users\yingyy\Desktop\Discoverability\Brand_Analysis.xlsx", sheet_name = 'CN')

In [36]:
result_2 = result[result['size']>=10]
result_2 = result_2[['Brand','Rank']].drop_duplicates()

In [37]:
result_2['size_binned'] = pd.cut(result_2['Rank'],bins)

In [39]:
result_2["size_binned"].value_counts(normalize= False,sort = False).to_excel(r"C:\Users\yingyy\Desktop\Discoverability\Brand_Analysis.xlsx", sheet_name = 'CN_Brank')